In [1]:
import tensorflow as tf

In [2]:
import rospy
import numpy as np
%load_ext autoreload
%autoreload 2

# roslaunch kinova_bringup kinova_robot.launch kinova_robotType:=j2n6s300 use_urdf:=true

# rosrun kinova_demo fingers_action_client.py -v j2n6s300 turn 1000 1000 1000

# pose

# $ rosrun kinova_demo pose_action_client.py -v  j2n6s300 mrad -- 0.07 -0.41 0.4 1.79 0.52 -0.22

# rosrun kinova_demo pose_action_client.py -v  j2n6s300 mdeg -- 0.3 0 0.4  -360 0 -90

# rosrun kinova_demo pose_action_client.py -v  j2n6s300 mrad -- 0.3 0 0.4  -3.13 0.0  -1.6

# joint1: 169.621444702
# joint2: 181.644241333
# joint3: 82.3455047607
# joint4: 330.047546387
# joint5: 97.5595321655
# joint6: -40.4018592834


# joint1: 264.667297363
# joint2: 180.805374146
# joint3: 110.492965698
# joint4: 302.359649658
# joint5: 143.923019409
# joint6: -64.330406189

#  init grasp joint position  original
# rosrun kinova_demo joints_action_client.py -v  j2n6s300 degree -- 169.6 181.6 82.3 330.0 97.6 -40.4 

# rosrun kinova_demo joints_action_client.py -v  j2n6s300 degree -- 259.6 181.6 82.3 330.0 97.6 -40.4 



# rostopic pub -r 100 /j2n6s300_driver/in/cartesian_velocity kinova_msgs/PoseVelocity "{twist_linear_x: 0.0, twist_linear_y: 0.0, twist_linear_z: 0.0, twist_angular_x: 0.0, twist_angular_y: 0.0, twist_angular_z: 10.0}" 

In [3]:
from helpers.gripper_action_client import set_finger_positions
from helpers.position_action_client import position_client, move_to_position
import tf.transformations as tft
#from helpers.transforms import current_robot_pose, publish_tf_quaterion_as_transform, convert_pose, publish_pose_as_transform
from helpers.covariance import generate_cartesian_covariance

the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'


In [4]:
rospy.init_node('ggcnn_test')

In [5]:
set_finger_positions([0, 0, 0])

fingers: 
  finger1: -6.0
  finger2: 6.0
  finger3: 0.0

In [6]:
move_to_position([0, -0.3 , 0.5], [1, 0, 0, 0.0078])

pose: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "j2n6s300_link_base"
  pose: 
    position: 
      x: 0.00162593228742
      y: -0.298371553421
      z: 0.497777849436
    orientation: 
      x: 0.999711039329
      y: 0.0077561265962
      z: 0.0114704340388
      w: 0.0196496688889

In [8]:
np.sqrt(1-0.99**2)

0.14106735979665894

In [7]:

MOVING = False  # Flag whether the robot is moving under velocity control.
CURR_Z = 0  # Current end-effector z height.

def robot_wrench_callback(msg):
    # Monitor wrench to cancel movement on collision.
    global MOVING
    if MOVING and msg.wrench.force.z < -2.0:
        MOVING = False
        rospy.logerr('Force Detected. Stopping.')


def robot_position_callback(msg):
    # Monitor robot position.
    global CURR_Z
    CURR_Z = msg.pose.position.z

    
def move_to_pose(pose):
    # Wrapper for move to position.
    p = pose.position
    o = pose.orientation
    move_to_position([p.x, p.y, p.z], [o.x, o.y, o.z, o.w])

def move_home():
    move_to_position([0, -0.64, 0.3], [-1, 0, 0, 0.001])

In [8]:

import kinova_msgs.msg
import kinova_msgs.srv
import std_msgs.msg
import std_srvs.srv
import geometry_msgs.msg
from helpers.transforms import current_robot_pose, publish_tf_quaterion_as_transform, convert_pose, publish_pose_as_transform

# Robot Monitors.
wrench_sub = rospy.Subscriber('/j2n6s300_driver/out/tool_wrench', geometry_msgs.msg.WrenchStamped, robot_wrench_callback, queue_size=1)
position_sub = rospy.Subscriber('/j2n6s300_driver/out/tool_pose', geometry_msgs.msg.PoseStamped, robot_position_callback, queue_size=1)


In [17]:
start_force_srv = rospy.ServiceProxy('/j2n6s300_driver/in/start_force_control', kinova_msgs.srv.Start)
stop_force_srv = rospy.ServiceProxy('/j2n6s300_driver/in/stop_force_control', kinova_msgs.srv.Stop)

move_home()

In [9]:
set_finger_positions([500, 500, 500])

fingers: 
  finger1: 504.0
  finger2: 492.0
  finger3: 504.0

In [9]:
# Get the positions.
msg = rospy.wait_for_message('/ggcnn/out/command', std_msgs.msg.Float32MultiArray)
d = list(msg.data)

In [10]:
d

[0.23334185779094696,
 -0.04684418439865112,
 0.6150000095367432,
 0.6724070310592651,
 80.011474609375,
 510.68060302734375]

In [11]:
grip_width = d[4]
# Convert width in pixels to mm.
# 0.07 is distance from end effector (CURR_Z) to camera.
# 0.1 is approx degrees per pixel for the realsense.
g_width = 2 * ((CURR_Z + 0.07)) * np.tan(0.1 * grip_width / 2.0 / 180.0 * np.pi) * 1000
# Convert into motor positions.
g = min((1 - (min(g_width, 70)/70)) * (6800-4000) + 4000, 5500)
g

4000

In [12]:
set_finger_positions([g, g, g])

fingers: 
  finger1: 3984.0
  finger2: 3978.0
  finger3: 3984.0

In [13]:
gp = geometry_msgs.msg.Pose()
gp.position.x = d[0]
gp.position.y = d[1]
gp.position.z = d[2]
gp.orientation.w = 1

# Convert to base frame, add the angle in (ensures planar grasp, camera isn't guaranteed to be perpendicular).
gp_base = convert_pose(gp, 'camera_depth_optical_frame', 'j2n6s300_link_base')
print(gp_base)

position: 
  x: -0.180753745872
  y: -0.35740727703
  z: -0.0825815676901
orientation: 
  x: 0.136745119604
  y: 0.989578750035
  z: -0.000428054066153
  w: 0.0451052825536


In [192]:
publish_pose_as_transform(gp_base, 'j2ns300_link_base', 'G', 0.5)

In [176]:
# I added
d[3] = 0 
gp_base.position.x = 0
gp_base.position.y = -0.64
gp_base.position.z = 0.02

# angle 
q = tft.quaternion_from_euler(-np.pi, 0, d[3])
gp_base.orientation.x = q[0]
gp_base.orientation.y = q[1]
gp_base.orientation.z = q[2]
gp_base.orientation.w = q[3]

print(gp_base)

position: 
  x: 0
  y: -0.64
  z: 0.02
orientation: 
  x: -1.0
  y: 0.0
  z: 0.0
  w: 6.123233995736766e-17


In [15]:
# Offset for initial pose.
initial_offset = 0.20
gp_base.position.z += initial_offset
gp_base

position: 
  x: -0.180753745872
  y: -0.35740727703
  z: 0.11741843231
orientation: 
  x: 0.136745119604
  y: 0.989578750035
  z: -0.000428054066153
  w: 0.0451052825536

In [18]:
# Disable force control, makes the robot more accurate.
stop_force_srv.call(kinova_msgs.srv.StopRequest())
move_to_pose(gp_base)

#g = 1000
set_finger_positions([g, g, g])

        the cartesian action timed-out


fingers: 
  finger1: 4008.0
  finger2: 4002.0
  finger3: 4008.0

In [179]:
# Reset the position
gp_base.position.z -= initial_offset
gp_base

position: 
  x: 0
  y: -0.64
  z: 0.02
orientation: 
  x: -1.0
  y: 0.0
  z: 0.0
  w: 6.123233995736766e-17

In [19]:

# Start force control, helps prevent bad collisions.
start_force_srv.call(kinova_msgs.srv.StartRequest())
stop_force_srv.call(kinova_msgs.srv.StopRequest())
# Flag to check for collisions.
MOVING = True

In [20]:
# Generate a nonlinearity for the controller.
cart_cov = generate_cartesian_covariance(0)
cart_cov

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [21]:
# Move straight down under velocity control.
velo_pub = rospy.Publisher('/j2n6s300_driver/in/cartesian_velocity', kinova_msgs.msg.PoseVelocity, queue_size=1)
while MOVING and CURR_Z - 0.02 > gp_base.position.z-0.03:
    dz = gp_base.position.z - CURR_Z - 0.03   # Offset by a few cm for the fingertips.
    MAX_VELO_Z = 0.10
    dz = max(min(dz, MAX_VELO_Z), -1.0*MAX_VELO_Z)

    v = np.array([0, 0, dz])
    vc = list(np.dot(v, cart_cov)) + [0, 0, 0]
    velo_pub.publish(kinova_msgs.msg.PoseVelocity(*vc))
    rospy.sleep(1/100.0)

KeyboardInterrupt: 

In [183]:
MOVING = False

# close the fingers.
rospy.sleep(0.1)
set_finger_positions([4800, 4800, 4800])
rospy.sleep(0.5)

In [184]:
# Move back up to initial position.
gp_base.position.z += initial_offset
gp_base.orientation.x = 1
gp_base.orientation.y = 0
gp_base.orientation.z = 0
gp_base.orientation.w = 0
move_to_pose(gp_base)

In [185]:
move_home()
set_finger_positions([0, 0, 0])

fingers: 
  finger1: 48.0
  finger2: 60.0
  finger3: 48.0